# Delta Practice

### File Read In

In [114]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

filepath4 = r'/Users/cartersocha/Downloads/instgramAccountCounts.xlsx'
igAccountDf = pd.read_excel(filepath4)

filepath5 = r'/Users/cartersocha/Downloads/redditCountTest.xlsx'
redditSubsDf = pd.read_excel(filepath5)

filepath6 = r'/Users/cartersocha/Downloads/redditCountFinal.txt'
#redditCommentsDf = pd.read_excel(filepath6)

filepath7 = r'/Users/cartersocha/Downloads/googleTvCount.xlsx'
googleTrendsDf = pd.read_excel(filepath7)

### Functions

In [115]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [116]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [117]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [118]:
def MergeDfs(mainDf, secondDf):

    merged = pd.merge(mainDf,secondDf, how='outer', on=['TvShow'])
    return merged

In [119]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf[columnName] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [120]:
def SumSocialMedia(sumDf,columnName):

    summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
    summarizedDf = pd.DataFrame(summarizedDf)

    return summarizedDf

### Data Transformations

##### Release Data

In [121]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=90)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [122]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'diffs')
instaHashDf['SocialMediaSource'] = 'InstagramHashtag'
instaHashDf['SocialMediaValue'] = instaHashDf['diffs']

iHashtagDf = SumSocialMedia(instaHashDf, 'TvShow')

iHashtagDf['SocialMediaValue'] = iHashtagDf['diffs']
iHashtagDf['SocialMediaSource'] = 'InstagramHashtag'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [123]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
dailyTweetDf['SocialMediaValue'] = dailyTweetDf['TweetCount'] + dailyTweetDf['RetweetCount']
dailyTweetDf['SocialMediaSource'] = 'Tweets'

iTweetCountDf = SumSocialMedia(dailyTweetDf, 'TvShow')
iTweetCountDf['SocialMediaSource'] = 'Tweets'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

redditCommentsDf = DateTimeConvert(redditCommentsDf,'runDate')

redCommentDf = RemoveData(releaseDf, redditCommentsDf)


iRedCommentDf = SumSocialMedia(redCommentDf, 'TvShow')
iRedCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']
iRedCommentDf['SocialMediaSource'] = 'RedditComments'

##### Instagram Account Data

In [124]:
igAccountDf = DateTimeConvert(igAccountDf,'RunDate')

igAccountDf = DiffMaker(igAccountDf,'IgAccountCounts','RunDate')


instaAccDf = RemoveData(releaseDf, igAccountDf)
instaAccDf = NegativeDiffs(instaAccDf, 'diffs')
instaAccDf['SocialMediaSource'] = 'InstagramAccount'
instaAccDf['SocialMediaValue'] = instaAccDf['diffs']

iAccountDf = SumSocialMedia(instaAccDf, 'TvShow')

iAccountDf['SocialMediaValue'] = iAccountDf['diffs']
iAccountDf['SocialMediaSource'] = 'InstagramAccount'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Subscriber Data

In [125]:
redditSubsDf = DateTimeConvert(redditSubsDf,'RunDate')

redditSubsDf = DiffMaker(redditSubsDf,'RedditSubs','RunDate')

redSubDf = RemoveData(releaseDf, redditSubsDf)
redSubDf = NegativeDiffs(redSubDf, 'diffs')
redSubDf['SocialMediaSource'] = 'RedditSubscribers'
redSubDf['SocialMediaValue'] = redSubDf['diffs']

iRedSubDf = SumSocialMedia(redSubDf, 'TvShow')
iRedSubDf['SocialMediaValue'] = iRedSubDf['diffs']
iRedSubDf['SocialMediaSource'] = 'RedditSubscribers'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Google Data

In [126]:
googleTrendsDf = DateTimeConvert(googleTrendsDf,'RunDate')

googDf = RemoveData(releaseDf, googleTrendsDf)
googDf = NegativeDiffs(googDf, 'GoogleValue')
googDf['SocialMediaSource'] = 'Google'
googDf['SocialMediaValue'] = googDf['GoogleValue']

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Join the Data

##### Time Value Datasets

In [127]:
instaAccDf = instaAccDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
redSubDf = redSubDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
dailyTweetDf = dailyTweetDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
instaHashDf = instaHashDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
googDf = googDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]

In [128]:
uberDf2 = pd.concat([instaAccDf,redSubDf,dailyTweetDf,instaHashDf,googDf])

uberDf3 = MergeDfs(uberDf2, releaseDf)
uberDf3 = uberDf3[uberDf3['Cancel'] != 0]
uberDf3.head(10)

,TvShow,RunDate,SocialMediaValue,SocialMediaSource,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,Cancel,90DayDate
0,ABlackLadySketchShow,2022-04-08,175.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
1,ABlackLadySketchShow,2022-04-09,341.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
2,ABlackLadySketchShow,2022-04-10,327.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
3,ABlackLadySketchShow,2022-04-11,222.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
4,ABlackLadySketchShow,2022-04-12,150.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
5,ABlackLadySketchShow,2022-04-13,71.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
6,ABlackLadySketchShow,2022-04-14,85.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
7,ABlackLadySketchShow,2022-04-15,73.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
8,ABlackLadySketchShow,2022-04-16,116.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07
9,ABlackLadySketchShow,2022-04-17,125.0,InstagramAccount,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07


##### Aggregation Datasets

igAccount = iAccountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
redditSub = iRedSubDf[['TvShow','SocialMediaValue','SocialMediaSource']]
tweetComments = iTweetCountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
igHashtag = iHashtagDf[['TvShow','SocialMediaValue','SocialMediaSource']]

##### Dataset Merging

nuberData = pd.concat([igAccount,redditSub,tweetComments,igHashtag],axis=0)
nuberData.head()

uberDf = MergeDfs(releaseDf, iHashtagDf)

uberDf = MergeDfs(uberDf, iTweetCountDf)

uberDf.head()

### Output - individuals & uber dataset

In [129]:
filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf3.to_csv(filepath2) 